In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from wordcloud import STOPWORDS
from wordcloud import WordCloud
from nltk.stem import PorterSteamer
from nltk.stem import WordNetLennatizer
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

ModuleNotFoundError: No module named 'pandas'

In [2]:
dataframe = pd.read_csv("data/ethos/Ethos_Dataset_Binary.csv", sep=';', header=0)
print(dataframe.size)
dataframe.tail()


NameError: name 'pd' is not defined